<a href="https://colab.research.google.com/github/Rajat-ops/classification/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_data, data_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [3]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [4]:
mnist_train

<DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [5]:
num_validation_samples = 0.1 * data_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1 * data_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [6]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

scaled_train_and_validation = mnist_train.map(scale)

In [8]:
buff_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation.shuffle(buffer_size=buff_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH = 100
train_data = train_data.batch(BATCH)

validation_data = validation_data.batch(num_validation_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [10]:
validation_inputs.shape

TensorShape([6000, 28, 28, 1])

In [19]:
# Outline the model

inputs_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                             
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                             ])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Optimizer and loss function

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
# Training

num_Epochs = 5

model.fit(train_data, epochs=num_Epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 11s - loss: 0.3370 - accuracy: 0.9040 - val_loss: 0.1624 - val_accuracy: 0.9535
Epoch 2/5
540/540 - 11s - loss: 0.1366 - accuracy: 0.9594 - val_loss: 0.1110 - val_accuracy: 0.9670
Epoch 3/5
540/540 - 10s - loss: 0.0957 - accuracy: 0.9707 - val_loss: 0.0932 - val_accuracy: 0.9712
Epoch 4/5
540/540 - 11s - loss: 0.0734 - accuracy: 0.9775 - val_loss: 0.0782 - val_accuracy: 0.9762
Epoch 5/5
540/540 - 11s - loss: 0.0574 - accuracy: 0.9823 - val_loss: 0.0685 - val_accuracy: 0.9817
